In [3]:
#Main library
import tensorflow as tf

#Helper libraries
import os, pickle, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
dir = ""
checkpoint_dir = dir + "model1/"
class_names = ['apple', 'apricot', 'avocado', 'banana', 'bell pepper', 'black berry', 'blueberry', 'cantaloupe', 'cherry', 
               'coconut', 'coffee', 'desert fig', 'eggplant', 'fig', 'grape', 'grapefruit', 'kiwi', 'lemon', 'lime', 'lychee', 
               'mango', 'orange', 'pear', 'pineapple', 'pomegranate', 'pumpkin', 'raspberry', 'strawberry', 'watermelon']

In [6]:
def load_model(dir):
  model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(29)
  ])

  model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

  latest = tf.train.latest_checkpoint(dir)
  model.load_weights(latest).expect_partial()
  return model

In [7]:
def predict(path, model):
  img = tf.keras.preprocessing.image.load_img(path, target_size=(180, 180))
  img_array = tf.keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)

  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])
  return score

In [8]:
model = load_model(checkpoint_dir)

In [33]:
img_path = input("path = ")
prediction = predict(img_path, model)
print("Got {} with {}% confidence.".format(class_names[np.argmax(prediction)], max(prediction) * 100))

Got strawberry with 62.6364631652832% confidence.


In [31]:
img_path = input("path = ")
prediction = predict(img_path, model)
predicted_class = class_names[np.argmax(prediction)]
confidences = [confidence.numpy() for confidence in prediction]

figs = confidences[class_names.index("fig")] + confidences[class_names.index("desert fig")]

if "fig" in predicted_class:
    predicted_class = "FIG"
    confidence = figs
else:
    confidence = confidences[class_names.index(predicted_class)] - figs
    predicted_class = "NOT FIG"

print("Got {} with {}% confidence.".format(predicted_class, str(confidence * 100)[:5]))

Got FIG with 100.0% confidence.
